In [4]:
# This code snippet represents a basic side-scrolling game developed using Python and the Pygame library.
# Here's a brief overview of the various components:

#a Constants and Setup: Initial configurations such as screen dimensions and colors.
#b Player Class: Defines the player's characteristics and movements, including jumping and respawn functionality.
#c Projectile Classes: Defines the behavior of projectiles for both the player and enemies.
#d Enemy Class: Defines the enemies, their movement, shooting mechanics, and health.
#e Game Class: Manages game levels (1-10), difficulty settings (easy,medium and hard), event handling, game logic, drawing, and game over screen interactions.

# How the Game Works
# Player Movement:
# - The player can move left or right and jump.
# - Shooting is controlled using the q (right) and w (up) keys.

# Enemies:
# - Move horizontally and shoot projectiles at intervals.
# - Number and shooting frequency are determined by the difficulty level.

# Game Levels and Difficulty:
# - The game progresses through levels, increasing the number of enemies with each level.
# - Three difficulty settings (Easy, Medium, Hard) influence enemy count and shooting frequency.

# Game Over Handling:
# - After losing all lives, a prompt appears to retry, choose a level, or exit.
# - Tkinter is used for simple input dialogs to get user input for difficulty and game over choices.

# Enhancements and Customization
# - Graphics and Sound: Implementing better graphics and sound effects can significantly enhance the game experience.
# - More Interactions: Adding power-ups, different enemy types, and varied level designs can make the game more engaging.
# - Optimizations: Implementing more efficient collision detection and handling large numbers of sprites better can improve performance.

In [6]:
import pygame
import random
import tkinter as tk
from tkinter import simpledialog

# Constants
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Player class with movement and lives
class Player(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((50, 50))
        self.image.fill((0, 255, 0))
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed = 5
        self.jump_speed = -15
        self.gravity = 1
        self.lives = 3
        self.is_jumping = False
        self.velocity_y = 0

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.speed
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.speed
        if keys[pygame.K_SPACE] and not self.is_jumping:
            self.velocity_y = self.jump_speed
            self.is_jumping = True
        
        self.velocity_y += self.gravity
        self.rect.y += self.velocity_y

        if self.rect.y >= HEIGHT - 100:
            self.rect.y = HEIGHT - 100
            self.is_jumping = False

    def respawn(self):
        self.rect.x = 50
        self.rect.y = HEIGHT - 100

# Projectile class for player bullets
class Projectile(pygame.sprite.Sprite):
    def __init__(self, x, y, direction):
        super().__init__()
        self.image = pygame.Surface((10, 10))
        self.image.fill((255, 0, 0))
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = 10
        self.direction = direction

    def update(self):
        if self.direction == "right":
            self.rect.x += self.speed
        elif self.direction == "up":
            self.rect.y -= self.speed
        
        if self.rect.x > WIDTH or self.rect.y < 0:
            self.kill()

# Enemy projectile class
class EnemyProjectile(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((10, 10))
        self.image.fill((0, 0, 255))
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = 5

    def update(self):
        self.rect.y += self.speed
        if self.rect.y > HEIGHT:
            self.kill()

# Enemy class with movement and health
class Enemy(pygame.sprite.Sprite):
    def __init__(self, x, y, game):
        super().__init__()
        self.image = pygame.Surface((50, 50))
        self.image.fill((255, 0, 0))
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = random.choice([-2, 2])
        self.health = 50
        self.shoot_delay = game.shoot_delay
        self.last_shot = pygame.time.get_ticks()
        self.game = game

    def update(self):
        self.rect.x += self.speed
        if self.rect.right > WIDTH or self.rect.left < 0:
            self.speed *= -1
        
        now = pygame.time.get_ticks()
        if now - self.last_shot > self.shoot_delay:
            self.shoot()
            self.last_shot = now

    def shoot(self):
        enemy_projectile = EnemyProjectile(self.rect.centerx, self.rect.centery)
        self.game.all_sprites.add(enemy_projectile)
        self.game.enemy_projectiles.add(enemy_projectile)

# Game class to manage levels and game logic
class Game:
    def __init__(self, difficulty, start_level=1):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Side-Scrolling Game")
        self.clock = pygame.time.Clock()
        self.running = True
        self.all_sprites = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()
        self.projectiles = pygame.sprite.Group()
        self.enemy_projectiles = pygame.sprite.Group()
        self.player = Player(50, HEIGHT - 100)
        self.all_sprites.add(self.player)
        self.level = start_level
        self.score = 0
        self.difficulty = difficulty
        self.set_difficulty()
        self.spawn_enemies()

    def set_difficulty(self):
        if self.difficulty == 'Easy':
            self.enemy_count = 3
            self.shoot_delay = 2000
        elif self.difficulty == 'Medium':
            self.enemy_count = 5
            self.shoot_delay = 1500
        elif self.difficulty == 'Hard':
            self.enemy_count = 7
            self.shoot_delay = 1000

    def spawn_enemies(self):
        for _ in range(self.enemy_count + self.level - 1):
            enemy = Enemy(random.randint(100, WIDTH - 100), random.randint(50, HEIGHT - 150), self)
            self.all_sprites.add(enemy)
            self.enemies.add(enemy)

    def run(self):
        while self.running:
            self.clock.tick(60)
            self.events()
            self.update()
            self.draw()
        pygame.quit()
        self.show_game_over_screen()

    def events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    projectile = Projectile(self.player.rect.centerx, self.player.rect.centery, "right")
                    self.all_sprites.add(projectile)
                    self.projectiles.add(projectile)
                elif event.key == pygame.K_w:
                    projectile = Projectile(self.player.rect.centerx, self.player.rect.centery, "up")
                    self.all_sprites.add(projectile)
                    self.projectiles.add(projectile)

    def update(self):
        self.all_sprites.update()

        # Check for player projectiles hitting enemies
        hits = pygame.sprite.groupcollide(self.projectiles, self.enemies, True, True)
        for hit in hits:
            self.score += 10

        # Check for enemy projectiles hitting player
        if pygame.sprite.spritecollide(self.player, self.enemy_projectiles, True) or pygame.sprite.spritecollide(self.player, self.enemies, False):
            self.player.lives -= 1
            if self.player.lives > 0:
                self.player.respawn()
            else:
                self.running = False

        # Progress to the next level if all enemies are defeated
        if not self.enemies:
            self.level += 1
            if self.level > 10:
                self.running = False  # Game ends after level 10
            else:
                self.spawn_enemies()

    def draw(self):
        self.screen.fill(WHITE)
        self.all_sprites.draw(self.screen)
        self.draw_text(f"Score: {self.score}", 22, BLACK, WIDTH / 2, 10)
        self.draw_text(f"Lives: {self.player.lives}", 22, BLACK, 50, 10)
        self.draw_text(f"Level: {self.level}", 22, BLACK, WIDTH - 100, 10)
        pygame.display.flip()

    def draw_text(self, text, size, color, x, y):
        font = pygame.font.SysFont('Arial', size)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        text_rect.midtop = (x, y)
        self.screen.blit(text_surface, text_rect)

    def show_game_over_screen(self):
        root = tk.Tk()
        root.withdraw()
        result = simpledialog.askstring("Game Over", "Retry (r), Choose Level (c), or Exit (e)?")
        root.destroy()
        if result == 'r':
            game = Game(self.difficulty, self.level)
            game.run()
        elif result == 'c':
            level = simpledialog.askinteger("Choose Level", "Enter level (1-10):")
            if level and 1 <= level <= 10:
                game = Game(self.difficulty, level)
                game.run()
            else:
                print("Invalid level")
        elif result == 'e':
            pygame.quit()
        else:
            print("Invalid choice")

def get_difficulty():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    difficulty = simpledialog.askstring("Difficulty", "Choose difficulty (Easy, Medium, Hard):")
    root.destroy()
    return difficulty

if __name__ == "__main__":
    difficulty = get_difficulty()
    if difficulty not in ['Easy', 'Medium', 'Hard']:
        difficulty = 'Easy'  # Default to Easy if input is invalid
    game = Game(difficulty)
    game.run()


Invalid level
